In [2]:
import json
import cv2
import numpy as np
import os
dir_name='20epoch_continue/'
all_dicts={}


for filename  in sorted(os.listdir(dir_name)):
    temp_list=[]
    with open (dir_name+filename) as f:
        filename=filename.replace(".txt","")
        for line in f:
            line=line.split()
            temp_list.append({'boxs':line[4:8],
            '3d_attribute':line[8:15],
            'group':-1})
        all_dicts[filename]=temp_list
        
def iou_2d(box1,box2):
    area1 = (box1[2]-box1[0])*(box1[3]-box1[1])
    area2 = (box2[2]-box2[0])*(box2[3]-box2[1])
    area_sum = area1 + area2
    
    #計算重疊座標 設重疊box座標為 [x1,y1,x2,y2]
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    if x1 >= x2 or y1 >= y2:
        return 0
    else:
        inter_area = (x2-x1)*(y2-y1)
    return inter_area/(area_sum-inter_area)
            

In [3]:
for filename  in sorted(os.listdir(dir_name)):
    with open (dir_name+filename) as f:
        filename=filename.replace(".txt","")
        for line in f:
            line=line.split()
            print(line[4:15])
        print(filename)
       
        

['730', '189', '998', '327', '1.48', '1.58', '3.78', '2.19', '1.67', '7.28', '-1.55']
['293', '190', '424', '230', '1.57', '1.67', '4.26', '-8.94', '1.72', '25.99', '-3.33']
['470', '182', '501', '197', '1.58', '1.63', '4.37', '-7.95', '1.08', '46.76', '1.41']
['500', '181', '523', '191', '1.43', '1.63', '4.02', '-9.30', '0.80', '69.64', '-1.60']
['370', '186', '448', '224', '1.54', '1.64', '3.93', '-8.42', '1.57', '30.65', '-1.39']
0000000000
['753', '192', '1028', '341', '1.46', '1.58', '3.73', '2.80', '1.44', '8.15', '-1.57']
['289', '193', '426', '233', '1.56', '1.66', '4.31', '-8.69', '1.79', '25.16', '-3.33']
['439', '181', '474', '215', '1.81', '0.67', '1.19', '-6.98', '1.50', '33.05', '-2.86']
['474', '185', '506', '199', '1.51', '1.64', '3.94', '-9.70', '1.34', '59.40', '1.63']
['396', '190', '449', '223', '1.45', '1.61', '3.91', '-9.33', '1.76', '36.36', '-1.53']
0000000001
['765', '190', '1093', '365', '1.47', '1.59', '3.79', '2.58', '1.39', '6.88', '-1.49']
['284', '196', '

In [5]:
for key,file in all_dicts.items():
    for index,info in enumerate(file):
        file[index]['boxs']=list(map(int,file[index]['boxs']))
        file[index]['3d_attribute']=list(map(float,file[index]['3d_attribute']))



In [8]:
def matching (first_frame,second_frame):
    for index,info in enumerate(first_frame):
        tempoverlap=0
        for second_frame_index,second_frame_info in enumerate(second_frame): 
            overlap=iou_2d(first_frame[index]['boxs'],second_frame[second_frame_index]['boxs'])
            if overlap>tempoverlap and overlap>0.5:
                tempoverlap=overlap
                #print(second_frame_index)
                first_frame[index]['group']=second_frame_index
                second_frame[second_frame_index]['group']=second_frame_index
        #print(tempoverlap)
        #print(index,second_frame_index)
def cal_consistency(first_frame_object,second_frame_object):
    dim1=first_frame_object[0]*first_frame_object[1]*first_frame_object[2]
    dim2=second_frame_object[0]*second_frame_object[1]*second_frame_object[2]
    if dim1>dim2:
        return dim1/dim2
    else:
        return  dim2/dim1
    
def dimconsistency(first_frame,second_frame):
    all_consistency=0
    match=0
    for index,info in enumerate(first_frame):
        if first_frame[index]['group'] == -1:
             continue
        for second_frame_index,second_frame_info in enumerate(second_frame):
            if first_frame[index]['group'] == second_frame[second_frame_index]['group']:
                match+=1
                consistency=cal_consistency(first_frame[index]['3d_attribute'][0:3],second_frame[second_frame_index]['3d_attribute'][0:3])
                #print(consistency)
                all_consistency+=consistency
                break
    return all_consistency/match
def orient_consistency(first_frame,second_frame):
    match=0
    orient_cons_list=[]
    for index,info in enumerate(first_frame):
        if first_frame[index]['group'] == -1:
            continue
        for second_frame_index,second_frame_info in enumerate(second_frame):
                if first_frame[index]['group'] == second_frame[second_frame_index]['group']:
                    match+=1
                    radius_diff=abs(first_frame[index]['3d_attribute'][6]-second_frame[second_frame_index]['3d_attribute'][6])
                    degree_diff=np.rad2deg(radius_diff)
                    orient_cons_list.append(degree_diff)
                    break
    # mean = sum(orient_cons_list) / len(orient_cons_list)
    # std_dev = (sum((x - mean) ** 2 for x in orient_cons_list) / len(orient_cons_list)) ** 0.5
    # orient_new_list=[]
    # orient_new_list = [x for x in orient_cons_list if (x > mean - (2 * std_dev) and x<mean+(2*std_dev)) ]
    return orient_cons_list

        


def iou_3d(box1,box2):
    '''box [x1,y1,z1,x2,y2,z2]'''
    area1=(box1[3]-box1[0])*(box1[4]-box1[1])*(box1[5]-box1[2])
    area2=(box2[3]-box2[0])*(box2[4]-box2[1])*(box2[5]-box2[2])
    area_sum =area1+area2
    #count overlap region assum overlapped box coordinate is [x1,y1,z1,x2,y2,z2]
    x1=max(box1[0],box2[0])
    y1=min(box1[1],box2[1])
    z1=max(box1[2],box2[2])
    x2=min(box1[3],box2[3])
    y2=max(box1[4],box2[4])
    z2=min(box1[5],box2[5])
    if x1>=x2 or y1<=y2 or z1 >= z2:
        return 0
    else :
        inter_area=(x2-x1)*(y2-y1)*(z2-z1)
    return inter_area/(area_sum-inter_area)
def corner_fine_tune(box_corner):
    if (box_corner[0]>box_corner[3]):
        box_corner[0],box_corner[3]=box_corner[3],box_corner[0]
    if (box_corner[2]>box_corner[5]):
        box_corner[2],box_corner[5]=box_corner[5],box_corner[2]
    return box_corner
def compute_3d_box_corner(h, w, l, x, y, z, yaw):
    R = np.array([[np.cos(yaw), 0, np.sin(yaw)], [0, 1, 0], [-np.sin(yaw), 0, np.cos(yaw)]])
    x_corners = [l/2,l/2,-l/2,-l/2,l/2,l/2,-l/2,-l/2]
    y_corners = [0,0,0,0,-h,-h,-h,-h]
    z_corners = [w/2,-w/2,-w/2,w/2,w/2,-w/2,-w/2,w/2]
    stack1=np.vstack([x_corners,y_corners,z_corners])
    corners_3d_cam = np.dot(R, np.vstack([x_corners,y_corners,z_corners]))
    corners_3d_cam+=np.vstack([x,y,z])
    box_corner=np.concatenate((corners_3d_cam[:,2],corners_3d_cam[:,4]),axis=0)
    box_corner =corner_fine_tune(box_corner)
    return box_corner

In [28]:
import numpy as np
print(type(all_dicts['0000000000'][1]['3d_attribute'][6]))
print(all_dicts['0000000000'][1]['3d_attribute'][6]-all_dicts['0000000001'][3]['3d_attribute'][6])
print(np.rad2deg(abs(all_dicts['0000000000'][4]['3d_attribute'][6]-all_dicts['0000000001'][4]['3d_attribute'][6])))

<class 'float'>
-4.91
2.8647889756541187


In [4]:
temp_key=0
consist_list=[]
for key,list in all_dicts.items():
    if temp_key==0:
        temp_key=key
        continue
    matching(all_dicts[temp_key],all_dicts[key])
    #dim_cons=dimconsistency(all_dicts[temp_key],all_dicts[key])
    orient_cons=orient_consistency(all_dicts[temp_key],all_dicts[key])
    consist_list.append(orient_cons)
    temp_key=key
print('-------')
consist_list=[item for sublist in consist_list for item in sublist ]
print(sum(consist_list)/len(consist_list))
mean = sum(consist_list) / len(consist_list)
std_dev = (sum((x - mean) ** 2 for x in consist_list) / len(consist_list)) ** 0.5
# for x in consist_list:
#     if not (x > mean - (2 * std_dev) and x<mean+(2*std_dev)):
#         print('False')
new_list = [x for x in consist_list if (x > mean - (2 * std_dev) and x<mean+(2*std_dev)) ]
new_list = [x for x in new_list if x<120]
print(new_list)
print(sum(new_list)/len(new_list))


-------
43.45349183819814
[9.167324722093166, 1.718873385392484, 2.8647889756541187, 0.0, 2.8647889756541316, 9.16732472209318, 5.729577951308237, 3.4377467707849423, 4.58366236104659, 26.356058576017865, 5.729577951308237, 8.02140913183152, 5.156620156177427, 2.291831180523295, 30.939720937064457, 24.06422739549456, 1.1459155902616476, 1.7188733853924585, 3.4377467707849423, 18.90760723931717, 1.1459155902616476, 6.875493541569885, 2.8647889756541187, 45.263665815335024, 2.291831180523295, 7.448451336700696, 27.501974166279513, 25.210142985756207, 6.875493541569872, 6.875493541569872, 0.5729577951308238, 8.021409131831533, 5.156620156177401, 2.291831180523295, 13.178029288008933, 1.7188733853924585, 8.594366926962344, 7.448451336700709, 4.58366236104659, 21.772396214971277, 49.27437038125079, 4.010704565915779, 37.81521447863433, 1.1459155902616476, 1.1459155902616476, 5.729577951308225, 20.62648062470963, 8.021409131831508, 1.1459155902616476, 19.48056503444798, 21.19943841984045, 1.

In [313]:

corner1=compute_3d_box_corner(1.57, 1.67, 4.26, -8.94, 1.72, 25.99, -3.33)

corner2=compute_3d_box_corner(1.56, 1.66, 4.31, -8.69, 1.79, 25.16, -3.33)

overlap=iou_3d(corner1,corner2)
print(overlap)


0.4161218879031358


In [320]:
firstframe_corner= compute_3d_box_corner(*(all_dicts['0000000000'][0]['3d_attribute']))
secondframe_corner=compute_3d_box_corner(*(all_dicts['0000000001'][0]['3d_attribute']))
overlap_3d=iou_3d(firstframe_corner,secondframe_corner)
print(overlap_3d)


0.2461412620029518
